# Build a Neural Network Model that Describes Images :)

In this Assignment you will get familiarized with two successful neural network models for vision and language, Convolutional Neural Networks(CNN) and Long Short Term Memory Networks(LSTM).

You will see how you can build a model that can generate captions for images putting a CNN and an LSTM together. The model you will build is based on what is proposed in [\showandtell]

We are going to use Python as the programming language and Theano as the library to implement the neural network operations. Please follow the Tutoral on building a simple Convolutional Neural Network with theano and python here to aquire the basics you need to do this assignment. 

## Convolutional Neural Networks

Convolutional Neural Networks are a type of feed forward neural networks that are inspired by the way visual information is processed in human brain and they were primarily proposed for image recognition tasks. 
In principle in feedforward neural networks the nodes are organized into layers and nodes of each layer can only have forward connections with the nodes of the next layer and backward connection with the nodes of the previous layer. In CNNs these connections between layers are kind of localized i.e. the connections do not  cover the whole input(image) instantly but only a part of it and they sweep the input(image) to cover all of it. This is what is called the Convolution step. 
There are four main layers in CNNs that you need to understand first in order to understand how they work. The main layer type is the convolution layer. 

Here you can see a visualisation of the Convolution step:
<img src=https://bamos.github.io/data/2016-08-09/padding_strides_transposed.gif />

Mathematecally what happens is that you have a set of connections(weights), say a KxM matrix W, these weight are applied at different location of the input image (boxes of size KxM) each pixel is multiplied by it corresponding weight and then they are all summed up. As you see in the picture, this results in a new matrix, which is called a feature map or a convolved feature



<i class="fa fa-lightbulb-o "></i>
<span>Having and NxN image, if we apply a KxK convolution matrix on it, what would be the size of the output?</span>

In the code below we read an image of a cat to into a 3d tensor. (A 2 dimensional image with 1 additional dimension for color  channels)

In [ ]:
import theano
from theano import tensor as T
from theano.tensor.nnet import conv2d
import numpy
import cv2
%matplotlib inline
rng = numpy.random.RandomState(23455)


 #open an image: read the image into pixels
img = cv2.resize(cv2.imread("cat.jpg"), (224, 224)).astype(np.float32)
img_ = img.transpose((2, 0, 1))

plt.figure(1)
imgplot1 = plt.imshow(img)



Then using theano we define the symbolic expressions for applying the convolution operation on an image input.

In [ ]:
input = T.tensor4("image_tensor")

w_shp = (1,3, 64, 64)
w_bound = numpy.sqrt(3 * 64* 64)
W = theano.shared( numpy.asarray(
            rng.uniform(
                low=-1.0 / w_bound,
                high=1.0 / w_bound,
                size=w_shp),
            dtype=input.dtype), name ='W')

conv_out = conv2d(input, W)
conv_step = theano.function([input],[conv_out])

And here we apply the convolution operation on our image.

In [ ]:
img2 = np.expand_dims(img_, axis=0)
print(img2.shape)
output = conv_step(img2)

plt.figure(1)
imgplot = plt.imshow(output[0][0][0])

<i class="fa fa-lightbulb-o "></i>
Change the weight matrix to contain different patters (Diagnal, Vertical/Horizental lines, Curves, Full square, different pattern for each color channel) and report how it affects the output.

<i class="fa fa-lightbulb-o "></i>
If you apply the convolution step multiple times how the result change at each step? In other words, the deeper you go?
Implement this and include the picture of the outputs for at least two conserquitive convolution steps in your report.

Similar to other feed forward neural networks, there is also the activation layer in CNNs. So after each convolution layer there is normally an activation layer. In most of the existing CNN architectures ReLU(Rectified Linear Unit) is used as the activation function. 

<img src="https://ml4a.github.io/images/figures/relu.png" />

The next layer after the convolution layer(includeing the activation function) in CNNs is the pooling step (also called downsampling). The main prupose of this step is to reduce the dimensionality of the feature maps while preserving the important features. There are different types of pooling i.e. Max pooling, Mean Pooling, Sum Pooling.

Here is a picture illustrating Max Pooling:
<img src="https://ujwlkarn.files.wordpress.com/2016/08/screen-shot-2016-08-10-at-3-38-39-am.png?w=988", width="300">


The code below, apply max pooling on the last output we have.
<i class="fa fa-lightbulb-o "></i>
Change the value of the down sample size and report its effect.

In [ ]:
from theano.tensor.signal.downsample import *

down_sample_size = 2
pooling_input = T.tensor4("pooling_input")
pooling_output = max_pool_2d( pooling_input, ds = (down_sample_size,down_sample_size) , mode='max', ignore_border=False)
                          
pool_step = theano.function([pooling_input],[pooling_output])
                          
                          
plt.figure(1)
imgplot = plt.imshow(pool_step(output[0])[0][0][0])

So far, you have seen what happens in the Convolutional Layer, Activation Layer and Pooling Layer. The last type of layers in CNNs is the Fully Connected Layer. In the fully connected layer all the input nodes are connected to all the output nodes. Thus, after one or more sequences of convolution-activation-pooling layers, the output of the newtwork is flattened into a one dimensional array, and there are one or more fully connected layers, mapping the flattened feature vector representation, normally for the purpose of classification.




<i class="fa fa-lightbulb-o "></i>
If you are going to design a CNN for an image recognition task, what are the parameters and hyper parameters that you need to decide about?

<i class="fa fa-lightbulb-o "></i>
Draw a high level structure of a CNN Network along with a description of its hyper parameters.
(For sure you can find plenty of CNN figures by googling, this excersise is to make sure you  completely understand the structure of these networks)

There are several different architectures for CNNs i.e. LeNet, AlexNet, GoogleNet, VGGNet, ResNets. In this assignment you are going to employ an instance of a VGGNet.



## Long Short Term Memory Networks

Recurrent neural networks are supposed to be a high level models of how sequential data is processed in human brain. In these networks, the output of the network at each step is computed not only based on the current input but also the previous state of the network.

LSTM networks are recurrent networks equipped with a memory compartment. At each LSTM block in addition to the nodes representing the state of the network, there are nodes( a vector) functioning as the memory of the network. Furthermore, there are gates controling how the state of the network, the memory content and the output should change at each timestep according the current input and the previous state. 



Here is a blog with very good explanation for the LSTM networks:
http://colah.github.io/posts/2015-08-Understanding-LSTMs/

## Here is a pricture of what we are trying to build in this asignment:

<img  src="tikz-figure0.png",width="400"/>

In [1]:
import theano
import theano.tensor as T
import numpy as np

from theanolm.matrixfunctions import orthogonal_weight
from lasagne.updates import sgd, apply_momentum

from src.VGG_16 import *
from src.util import * 

Using Theano backend.


In [2]:
VOCAB = ['#', 'red','green','blue','square','circle','$']
Dic = {'#':0, 'red':1, 'green':2, 'blue':3, 'square':4, 'circle':5, '$':6}
ONE_HOT_VECS = np.eye(len(VOCAB))


Here we read the images from the 6 categories:

In [3]:
red_squares, thumb_red_squares = load_images_from_folder("red_square")
red_square_label = np.asarray([(ONE_HOT_VECS[Dic["red"]]),(ONE_HOT_VECS[Dic["square"]])])
red_square_labels =  np.repeat([red_square_label], len(red_squares),axis=0)

green_squares, thumb_green_squares = load_images_from_folder("green_square")
green_square_label = np.asarray([(ONE_HOT_VECS[Dic["green"]]),(ONE_HOT_VECS[Dic["square"]])])
green_square_labels =  np.repeat([green_square_label], len(green_squares),axis=0)

blue_squares, thumb_blue_squares = load_images_from_folder("blue_square")
blue_square_label = np.asarray([(ONE_HOT_VECS[Dic["blue"]]),(ONE_HOT_VECS[Dic["square"]])])
blue_square_labels =  np.repeat([blue_square_label], len(blue_squares),axis=0)

red_circles, thumb_red_circles = load_images_from_folder("red_circle")
red_circle_label = np.asarray([(ONE_HOT_VECS[Dic["red"]]),(ONE_HOT_VECS[Dic["circle"]])])
red_circle_labels =  np.repeat([red_circle_label], len(red_circles),axis=0)

green_circles, thumb_green_circles = load_images_from_folder("green_circle")
green_circle_label = np.asarray([(ONE_HOT_VECS[Dic["green"]]),(ONE_HOT_VECS[Dic["circle"]])])
green_circle_labels =  np.repeat([green_circle_label], len(green_circles),axis=0)

blue_circles, thumb_blue_circles = load_images_from_folder("blue_circle")
blue_circle_label = np.asarray([(ONE_HOT_VECS[Dic["blue"]]),(ONE_HOT_VECS[Dic["circle"]])])
blue_circle_labels =  np.repeat([blue_circle_label], len(blue_circles),axis=0)

Let's see what happens if we apply the VGG network on our data!

First we instantiate a VGG network and load it with the weights of a previously trained model. If you want to train the network your self it would take days. Running the below code will give you a summary of the VGG network architecture you are about to use.

<i class="fa fa-lightbulb-o "></i>
How many Convolution Layers does this VGGNet have? How many filters does it have at each layer? what are the dimensions of the filters at each layer? (Answer the last question only for the first two layers)

<i class="fa fa-lightbulb-o "></i>
How removing the pooling layers would affect the size of the network?

In [4]:
vgg = VGG_16('vgg16_weights.h5')
_sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
vgg.model.compile(optimizer=_sgd, loss='categorical_crossentropy')

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           zeropadding2d_input_1[0][0]      
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 224, 224)  36928       zeropadding2d_2[0][0]            
___________________________________________________________________________________________

The VGGNet we are using is trained on the ImageNet dataset to classify images into 1000 concepts. In this caption generator model we are going to use the one before last layer of the network as kind of conceptual representation of images. This mean we have a high dimensional vector representation of images that is quiet hard to qualitatively analyse. There are several methods to help us represent the high dimensional data in lower dimensions and makes it possible for us to plot them. One of these methods which is very popular currently is the t-SNE model. if you are curious you can read more about this method in here [].

Now we will use the t-SNE method to see how the representation of different shape-color combinations is distributed in the space.

<i class="fa fa-lightbulb-o "></i>
Do you think these image represention are good enough so that the LSTM network can learn to assign proper captions to our images?

In [5]:
thumbs = []
thumbs.extend(thumb_red_squares)
thumbs.extend(thumb_green_squares)
thumbs.extend(thumb_blue_squares)
thumbs.extend(thumb_red_circles)
thumbs.extend(thumb_green_circles)
thumbs.extend(thumb_blue_circles)

shapes = []
shapes.extend(red_squares)
shapes.extend(green_squares)
shapes.extend(blue_squares)
shapes.extend(red_circles)
shapes.extend(green_circles)
shapes.extend(blue_circles)

labels = []
labels.extend(red_square_labels)
labels.extend(green_square_labels)
labels.extend(blue_square_labels)
labels.extend(red_circle_labels)
labels.extend(green_circle_labels)
labels.extend(blue_circle_labels)

items = np.arange(len(labels))
np.random.shuffle(items)
thumb_images = []
representations = []

for i in items[0:100]:
    representations.append(vgg.get_representation(shapes[i])[0])
    thumb_images.append(thumbs[i])


In [6]:
t0 = time()

print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
X_tsne = tsne.fit_transform(representations)
plot_embedding(X_tsne, thumb_images,
                   "t-SNE embedding of the the flattend layer of the VGG Network (time %.2fs)" %
                   (time() - t0))
plt.show()

Computing t-SNE embedding


<i class="fa fa-lightbulb-o "></i>
Look into the define_network function in the ImageCaptionGenerator class defined below. The lstm_step is where the symbolic expression of the LSTM block is coded. Draw a graph that shows how the components of the LSTM block are related (Draw the computation graph of the LSTM block). 

<i class="fa fa-lightbulb-o "></i>
Look into the whole model and determine all of the parameters and explain their role in the network. which one of them change during the training process.

In [7]:


class ImageCaptionGenerator(object):
    
    def __init__(self, 
                 image_rep_dim, 
                 lstm_input_dim, 
                 lstm_hidden_dim, 
                 output_dim, 
                 learning_rate,vocab_size):
        self.image_rep_dim = image_rep_dim
        self.lstm_input_dim = lstm_input_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.vocab_size = vocab_size
        self.init_params()
        
        
    def init_params(self):
        WordEmbedding = orthogonal_weight(self.lstm_input_dim, self.output_dim,scale=0.01)
        ImageEmbedding = orthogonal_weight(self.lstm_input_dim, self.image_rep_dim,scale=0.01)

        self.WordEmbedding = theano.shared(value=WordEmbedding, name="WordEmbedding", borrow="True")
        self.ImageEmbedding = theano.shared(value=ImageEmbedding, name="ImageEmbedding", borrow="True")

        U_input = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        U_forget = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        U_output = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        W_input = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        W_forget = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        W_output = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        U = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        W = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        self.W = theano.shared(value=W, name="W" , borrow="True")
        self.U = theano.shared(value=U, name="U" , borrow="True")

        self.W_input = theano.shared(value=W_input, name="W_input" , borrow="True")
        self.U_input = theano.shared(value=U_input, name="U_input" , borrow="True")

        self.W_output = theano.shared(value=W_output, name="W_output" , borrow="True")
        self.U_output = theano.shared(value=U_output, name="U_output" , borrow="True")

        self.W_forget = theano.shared(value=W_forget, name="W_forget" , borrow="True")
        self.U_forget = theano.shared(value=U_forget, name="U_forget" , borrow="True")


      


        O_w = orthogonal_weight(self.output_dim, self.lstm_hidden_dim)
        self.O_w = theano.shared(value=O_w, name="O_w" , borrow="True")

        
        bias_input = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias_forget = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias_output = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        
        self.bias = theano.shared(value=bias, name="bias", borrow="True")
        self.bias_input = theano.shared(value=bias_input, name="bias_input", borrow="True")
        self.bias_output = theano.shared(value=bias_output, name="bias_output", borrow="True")
        self.bias_forget = theano.shared(value=bias_forget, name="bias_forget", borrow="True")

        
        self.params = [self.U_input, self.U_forget, self.U_output, self.W_input, self.W_forget, self.W_output,
                       self.U, self.W,
                       self.O_w, self.WordEmbedding, self.ImageEmbedding,
                       self.bias, self.bias_input, self.bias_output, self.bias_forget]
    
    def define_network(self):
        
        Y = T.matrix('output')
        H = T.vector('init_state')
        
        def lstm_step(prev_output,prev_state,prev_content):
            input_gate = T.nnet.hard_sigmoid(T.dot(( self.U_input),prev_output) + T.dot(self.W_input,prev_state) + self.bias_input)
            forget_gate = T.nnet.hard_sigmoid(T.dot(( self.U_forget),prev_output) + T.dot(self.W_forget,prev_state)+ self.bias_forget)
            output_gate = T.nnet.hard_sigmoid(T.dot((self.U_output),prev_output) + T.dot(self.W_output,prev_state)+ self.bias_output)



            stabilized_input = T.tanh(T.dot((self.U),prev_output) + T.dot(self.W,prev_state) + self.bias)
            c = forget_gate * prev_content + input_gate * stabilized_input
            s = output_gate * T.tanh(c) 
            
            output = T.nnet.softmax(T.dot(self.O_w,s))[0]
            
            next_input = T.dot(self.WordEmbedding,output)
            
            return [output,next_input,s,c],  theano.scan_module.until(np.argmax(output) > (self.vocab_size - 2))
        
        
        [self.output,_,self.hidden_state, self.memory_content], updates = theano.scan(
            lstm_step,
            truncate_gradient=-1,
            n_steps= 2,
            outputs_info=[None,dict(initial = T.dot(self.ImageEmbedding,H)),
                          dict(initial= T.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)),
                          dict(initial=T.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)),
                          ])
                                                          
        
        self.predict = theano.function([H],[self.output])

        params = self.params

        padded_output = T.zeros(Y.shape) 
        padded_output = T.set_subtensor(padded_output[0:self.output.shape[0],:],self.output)
        padded_output = T.set_subtensor(padded_output[self.output.shape[0]:, :], T.eye(self.output_dim)[self.output_dim - 1])
        cost = T.sum(T.nnet.categorical_crossentropy(padded_output + 0.000001,Y))
        updates_sgd = sgd(cost, params, learning_rate=self.learning_rate)
        updates = apply_momentum(updates_sgd, params, momentum=0.9)
        self.backprop_update = theano.function([H, Y], [cost], updates=updates)


In [8]:
print(representations[0].shape[0])

cp = ImageCaptionGenerator(image_rep_dim = representations[0].shape[0], 
                 lstm_input_dim = 1000, 
                 lstm_hidden_dim = 100, 
                 output_dim = len(VOCAB), 
                 learning_rate = 0.0001,
                 vocab_size = len(VOCAB))

cp.define_network()

4096


In [14]:
predicted_label = cp.predict(representations[0])
print(predicted_label[0])
print(get_string(predicted_label[0],VOCAB))

[[ 0.0828779   0.25230742  0.13702966  0.28405258  0.09512977  0.0890635
   0.05953917]
 [ 0.11802917  0.17841165  0.14573647  0.29679263  0.10136519  0.09404694
   0.06561795]]
blue blue


### Training the Model
Devide your data into train and test sets explain what are the criteria that should be considered in making test and train sets and report accuracies for train and test sets for each of the below cases. (Attention! Remove the backpropagation update line in test time.)

<i class="fa fa-lightbulb-o "></i>
train the network to label images based on their color

<i class="fa fa-lightbulb-o "></i>
train the network to label images based on their shape

<i class="fa fa-lightbulb-o "></i>
Now train the network to label images using the combinations of color and shape

<i class="fa fa-lightbulb-o "></i>
what happens if you completely remove one of the image-shape combination collections (say all blue circles) from the training set in each of these conditions. Is the network able to label it in the test time? How can you explain this?





In [17]:
shuffled_indexes = np.random.permutation(len(shapes))
train_indexes = shuffled_indexes[0:4000]
test_indexes = shuffled_indexes[4000:]
print(train_indexes)
print(test_indexes)

[1049  772 3440 ..., 5458 1656 1508]
[1359 3152  484 ...,  426 1815   33]


In [ ]:
number_of_epochs = 1
for e in np.arange(number_of_epochs):
    np.random.shuffle(train_indexes)
    corrects = 0
    for i in train_indexes:
        rep = vgg.get_representation(shapes[i])[0]
        predicted_label = cp.predict(rep)
        p_str = get_string(predicted_label[0],VOCAB)
        t_str = get_string(labels[i],VOCAB)
        print("Predicted: "+p_str+" Target:"+t_str)
        
        #remove this line when you want to test the model.
        cp.backprop_update(rep, np.asarray(labels[i],dtype="float32"))
        if p_str == t_str:
            corrects += 1
    accuracy = corrects / len(train_indexes)
    print("accuracy at epoch# "+str(e)+": "+str(accuracy))
    

Predicted: blue blue Target:blue square
Predicted: blue blue Target:red circle
Predicted: square blue Target:blue square
Predicted: blue blue Target:blue square
Predicted: blue square Target:blue circle
Predicted: blue blue Target:blue square
Predicted: blue blue Target:green square
Predicted: blue blue Target:green square
Predicted: blue square Target:green square
Predicted: square square Target:blue square
Predicted: green square Target:green square
Predicted: green square Target:red square
Predicted: green square Target:red square
Predicted: green square Target:blue square
Predicted: red square Target:blue circle
Predicted: green square Target:blue square
Predicted: green square Target:red square
Predicted: blue square Target:red square
Predicted: red square Target:green square
Predicted: blue circle Target:blue circle
Predicted: red square Target:red square
Predicted: blue square Target:blue square
Predicted: red square Target:green square
Predicted: red square Target:red square
Pr

#### Learning Rate:

You have seen in the previous assignment that learning rate can affect the learning process of a network. 
<i class="fa fa-lightbulb-o "></i>
Explain how the leaning rate affects the caption generator model (include learning curve graphs in your report.)

<i class="fa fa-lightbulb-o "></i>
Is it biologically plausible to have a fixed learning rate through the training phase?

<i class="fa fa-lightbulb-o "></i>
What is a reasonoble way to dynamically change the learning rate in the training phase?

<i class="fa fa-lightbulb-o "></i>
Find the simplest solution to this and implement it for the caption generator model. How does it affect the performance of the model?

#### Loss function:
<i class="fa fa-lightbulb-o "></i>
What is the loss function of the LSTM network in this assignment? Can you think of an alternative way to calculate loss? Test your proposed loss function and report how it affects the model.

#### Regularization:
<i class="fa fa-lightbulb-o "></i>
One of the main issues with deep neural networks is that they easily overfit the data. Do you think it is possible to avoid this issue to some degree by the decisions you make about the size of the network and the training data? Explain your answer and Test your solution if you think it is possible.


One of the solutions to help the nrual networks generalize and avoid overfitting is adding some regularizer terms to the loss function. It is called L1/L2 regularization. Where you add L1, L2 or both to your loss function. L2 is the sum of the square of the weights, while L1 is just the sum of the weights.

<i class="fa fa-lightbulb-o "></i>
How L1/L2 regularization helps the network to avoid overfitting?

<i class="fa fa-lightbulb-o "></i>
Add the L2 regularizaion to the LSTM model and report how it affects the result.



#### Analysing the Model
<i class="fa fa-lightbulb-o "></i>
Draw a graph that shows which output nodes of the VGGNet that determine the colors and the shapes of the images.

<i class="fa fa-lightbulb-o "></i>
Add another layer to the LSTM Network, how does it affect the performance of the model?

<i class="fa fa-lightbulb-o "></i>
Choose either the LSTM or CNN, and explain how backpropagation works in these models.